#### 필요 라이브러리

In [ ]:
# 데이터 처리
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import json
import itertools
import cv2
from torch.utils.data import DataLoader

# ViT
from timm import create_model, list_models
from types import SimpleNamespace

# GPT
from transformers import GPT2LMHeadModel, GPT2TokenizerFast, get_linear_schedule_with_warmup

# 데이터 증강
import albumentations as A
from albumentations.pytorch import ToTensorV2 # 데이터 -> PyTorch 텐서로 변환
from PIL import Image # 이미지 처리
from pathlib import Path # 파일 경로 관리
from sklearn.model_selection import train_test_split
from torch.cuda.amp import GradScaler, autocast
from tqdm.auto import tqdm # 진행 상태 표시
import gc

# 최종 예측
import subprocess
import matplotlib.pyplot as plt
from IPython.display import display, Video

### Fine-tuning the Model

#### Loading the Data

In [ ]:
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token

In [ ]:
### 데이터 불러오기
base_path = '/content/drive/MyDrive/DATA_최종'
images_path = os.path.join(base_path, 'Images')
labels_path = os.path.join(base_path, 'Labels')

In [ ]:
### 영상 프레임 가져오기

# 행동 폴더 이름 -> action_classes
action_classes = os.listdir(images_path)
print(f"고냥이의 행동 {len(action_classes)}개 있어요 ฅ^._.^ฅ")

# 각 폴더로 들어가서 - 비디오 이름들 가져오고 - 그 비디오 폴더 안으로 다시 들어가서 - 비디오 프레임들 가져오기 (200개 정도만?)
frames = {}
videos = {}

for action in action_classes:
    videos[action] = [] # 초기화
    frames[action] = [] # 초기화

    images_path_behav = os.path.join(images_path, action) # 각 폴더 경로
    video_names = os.listdir(images_path_behav) # 비디오 이름들 가져오기

    count = 0
    for video in video_names:
        if count >= 100:
          break
        frames_path = os.path.join(images_path_behav, video) # 각 영상 경로
        frame_names = os.listdir(frames_path) # 각 영상의 프레임 이름들

        if not frame_names:
          print(f"경고: {frames_path} 폴더가 비어 있습니다. 건너뜁니다.")
          continue

        one_video_frames = [os.path.join(frames_path, fname) for fname in frame_names]
        frames[action].append(one_video_frames)
        videos[action].append(video)
        count += 1

In [ ]:
### 영상 텍스트 가져와서 labels에 저장
labels = []
for action in videos.keys(): # 각 행동별로
    for video in videos[action]: # 각 비디오별로
        label_path = os.path.join(labels_path, action, video) + '.json' # json 파일 경로 설정
        try:
          with open(label_path, 'r') as f:
              json_file = json.load(f)
              labels.append("고양이가 " + json_file['metadata']['owner']['situation'] + " " + json_file['metadata']['action']) # situation, action 불러오기
        except Exception as e:
          print(f"{label_path} 에서 오류 발생: {e}")


In [ ]:
!pip install deepl

In [ ]:
import deepl

AUTH_KEY = "##########"
translator = deepl.Translator(AUTH_KEY)

batch_size = 10
translated_labels = []

for i in range(0, len(labels), batch_size):
    batch = labels[i:i+batch_size]
    translated_results = translator.translate_text(batch, source_lang="KO", target_lang="EN-US")
    translated_labels.extend([result.text for result in translated_results])

In [ ]:
all_videos = list(itertools.chain(*videos.values()))
all_frames = list(itertools.chain(*frames.values()))

In [ ]:
### 데이터프레임으로
kitties = pd.DataFrame({'video_names' : all_videos, 'videos' : all_frames, 'caption' : translated_labels})
kitties.head(2)

In [ ]:
### 잘 로드되었는지 확인
image_path = kitties['videos'][5][13]
image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
plt.imshow(image)

#### Dataset, DataLoader

In [ ]:
class Dataset:
    def __init__(self, df, tokenizer, transform):
        self.df = df
        self.tokenizer = tokenizer
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        sample = self.df.iloc[idx, : ]
        frame_paths = sample['videos']
        caption = sample['caption']

        # 이미지 불러오기
        frames = []
        for frame_path in frame_paths:
            image = cv2.imread(frame_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = np.array(image, dtype = np.uint8)
            image = self.transform(image = image)["image"] # 바뀐 애들 중에서 image만 떼어오기
            frames.append(image)

        frames_tensor = torch.stack([torch.as_tensor(frame, dtype = torch.float32).clone().detach() for frame in frames])

        # 텍스트 토큰화
        caption = f"{caption}<|endoftext|>"
        input_ids = self.tokenizer(
            caption,
            truncation=True,
            return_tensors = "pt")['input_ids'].squeeze(0) # squeeze by gpt
        labels = input_ids.clone()
        labels[ :-1] = input_ids[1: ]
        return frames_tensor, input_ids, labels

In [ ]:
train_kitties, val_kitties = train_test_split(kitties, test_size=0.1)

In [ ]:
train_kitties = Dataset(train_kitties, tokenizer, transform)
val_kitties = Dataset(val_kitties, tokenizer, transform)

In [ ]:
# 영상들의 프레임 개수 리스트 - 얼마로 패딩할지 정하기 위함 (평균 프레임 개수로 정했습니당)
frames_num = []
for i in range(len(kitties)):
    frames_num.append(len(kitties.loc[i, 'videos']))

video_padding = np.mean(frames_num)

In [ ]:
def collate_fn(batch):
    frames, input_ids, labels = zip(*batch)

    # 이미지 패딩
    padding_size = int(video_padding)
    padded_frames = []

    for video in frames:
        current_length = video.shape[0]

        if current_length > padding_size: # 패딩보다 길다면
            video = video[ :padding_size] # 잘라내고
        else:
            pad_size = padding_size - current_length # 짧으면 부족한 만큼 채우기
            pad_tensor = torch.zeros((pad_size, *video.shape[1: ]))
            video = torch.cat([video, pad_tensor], dim = 0)

        padded_frames.append(video)

    padded_frames_together = torch.stack(padded_frames)

    # 텍스트 패딩
    input_ids = torch.nn.utils.rnn.pad_sequence(input_ids, batch_first = True)
    labels = torch.nn.utils.rnn.pad_sequence(labels, batch_first = True, padding_value = -100) # 나중에 loss 계산할 때 무시하라고 -100으로 처리

    return padded_frames_together, input_ids, labels

In [ ]:
### DataLoader 설정
train_dataloader = DataLoader(
    train_kitties,
    batch_size = 4,
    shuffle = True,
    num_workers = 1,
    collate_fn = collate_fn)

val_dataloader = DataLoader(
    val_kitties,
    batch_size = 4,
    shuffle = False,
    num_workers = 1,
    collate_fn = collate_fn)